In [1]:
import pathlib

import duckdb
import pandas as pd

In [2]:
result_path = pathlib.Path("../../2.cellprofiling/results/C4-2/").resolve(strict=True)
# get a list of all parquets in the directory
parquet_files = list(result_path.glob("*.parquet"))
parquet_files.sort()

In [3]:
pd.options.display.max_columns = None

In [8]:
feature_types_dict = {
    "Organoid": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
    "Cell": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
    "Nuclei": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
    "Cytoplasm": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
}
for file in parquet_files:
    for compartment in feature_types_dict.keys():
        for feature_type in feature_types_dict[compartment].keys():
            if compartment in file.name and feature_type in file.name:
                feature_types_dict[compartment][feature_type].append(file)
feature_types_dict.keys()

dict_keys(['Organoid', 'Cell', 'Nuclei', 'Cytoplasm'])

In [9]:
# create a record for each compartment
merged_df_dict = {
    "Organoid": [],
    "Cell": [],
    "Nuclei": [],
    "Cytoplasm": [],
}

In [ ]:
df = pd.read_parquet(feature_types_dict["AreaSize_Shape"][0])
df_dict = {
    "organoid": [],
    "nuclei": [],
    "cell": [],
    "cytoplasm": [],
}
for areasizeshape_df in feature_types_dict["AreaSize_Shape"]:
    df = pd.read_parquet(areasizeshape_df)
    # rename any column that contain object id to "ObjectID"
    for col in df.columns:
        if "object_id" in col:
            df.rename(columns={col: "ObjectID"}, inplace=True)
        elif "image_set" in col:
            continue
        else:
            # prepend the column name with the feature type
            df.rename(columns={col: f"AreaSizeShape_{col}"}, inplace=True)
    if "Organoid" in areasizeshape_df.stem:
        df_dict["organoid"].append(df)
    elif "Nuclei" in areasizeshape_df.stem:
        df_dict["nuclei"].append(df)
    elif "Cell" in areasizeshape_df.stem:
        df_dict["cell"].append(df)
    elif "Cytoplasm" in areasizeshape_df.stem:
        df_dict["cytoplasm"].append(df)

# merge all of the organoid dataframes into one by ObjectID and image_set with duckdb
con = duckdb.connect()
dfs = {}
for i, df in enumerate(df_dict["organoid"]):
    dfs[f"{i}"] = df

KeyError: 'AreaSize_Shape'

In [75]:
df

,ObjectID,AreaSizeShape_Organoid_Mito_VOLUME,AreaSizeShape_Organoid_Mito_CENTER.X,AreaSizeShape_Organoid_Mito_CENTER.Y,AreaSizeShape_Organoid_Mito_CENTER.Z,AreaSizeShape_Organoid_Mito_BBOX.VOLUME,AreaSizeShape_Organoid_Mito_MIN.X,AreaSizeShape_Organoid_Mito_MAX.X,AreaSizeShape_Organoid_Mito_MIN.Y,AreaSizeShape_Organoid_Mito_MAX.Y,AreaSizeShape_Organoid_Mito_MIN.Z,AreaSizeShape_Organoid_Mito_MAX.Z,AreaSizeShape_Organoid_Mito_EXTENT,AreaSizeShape_Organoid_Mito_EULER.NUMBER,AreaSizeShape_Organoid_Mito_EQUIVALENT.DIAMETER,AreaSizeShape_Organoid_Mito_SURFACE.AREA,image_set
0,1,20948726.0,663.314682,552.593818,15.26746,33991848.0,124,1152,45,1047,0,33,0.616287,1,342.021178,NaN,C4-2


In [74]:
def duckdb_merge(dfs: dict, key: str = "id", join_type: str = "INNER") -> pd.DataFrame:
    """
    Dynamically merges multiple DataFrames using DuckDB on a common key.

    Parameters:
    - dfs (dict): Dictionary of {name: DataFrame}
    - key (str): The common key to join on
    - join_type (str): Join type: INNER, LEFT, RIGHT, FULL

    Returns:
    - Merged pandas DataFrame
    """
    con = duckdb.connect()

    # Register DataFrames in DuckDB
    for name, df in dfs.items():
        con.register(name, df)

    # Start building the SQL query
    table_names = list(dfs.keys())
    base = table_names[0]
    query = f"SELECT * FROM {base} "

    for tbl in table_names[1:]:
        query += f"{join_type} JOIN {tbl} USING({key}) "

    return con.execute(query).fetchdf()


duckdb_merge(
    dfs,
    key="ObjectID",
    # join_type="INNER",
)

ParserException: Parser Error: syntax error at or near "0"